In [ ]:
# | default_exp ds/config
# | export
from pydantic import (
    BaseModel,
    StringConstraints,
    model_validator,
)
from datetime import datetime, date

from typing import Annotated

from space_analysis.plot.config import FigureExtraOptions

In [ ]:
# | export
class OutputConfig(BaseModel):
    path: str = None
    filename: str = None

    formats: list[str] = list()
    display: bool = False

    figure: dict = dict()
    """Figure options"""

    figure_extra: FigureExtraOptions = FigureExtraOptions()
    """Extra figure options"""
    
class PanelConfig(BaseModel):

    timerange: list[date | datetime] = None
    
    ds: str = None
    id: str = None
    name: str = None
    units: str = None
    labels: list[str] = list()
    
    satellite: Annotated[str, StringConstraints(to_lower=True)] = None
    instrument: Annotated[str, StringConstraints(to_lower=True)] = None
    datatype: str = None
    probe: str = None
    
    # process: ProcessConfig = ProcessConfig()
    # tplot: TplotConfig = TplotConfig()

In [ ]:
# | export
class Config(BaseModel):
    panels: list[PanelConfig]
    timerange: list[datetime | date] = None
    output: OutputConfig = None
    backend: str = None
    
    @model_validator(mode='before')
    def set_default_timerange(cls, values):
        timerange = values.get('timerange', None)
        panels = values.get('panels', [])
        
        if timerange:
            for panel in panels:
                if not panel.get('timerange'):
                    panel['timerange'] = timerange
        return values